# Sedov Blast Wave

Figure 5.4 from Chapter 5 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Analytic solution of the Sedov-Taylor Blastwave following Kamm, J.R., 2000, _Evaluation of the 
Sedov-von Neumann-Taylor Blast Wave Solution_, LANL Report LA-UR-00-6055, LANL Applied Physics Division.

The three model parameters in Kamm's analytic solutions are:
 * $j$ = dimensionality, j={1,2,3}={planar,cylindrical,spherical}
 * $\omega$ = the power-law index of the initial density distribution $\rho_0(r)=Ar^{-\omega}$ where the  requirement that the mass (integral of $\rho$) is finite means $\omega < j$
 * $\gamma$ = the adiabatic index

While the parameters chosen for this specific calculation is for the ISM/IGM book, but it may be used more
generally to explore Sedov blast waves.

In [ ]:
%matplotlib inline

import math
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig5_4' 

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'
# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Sedov-Taylor Blast Wave calculation

Implementation of the Kamm 2000 analytic form of the generalized Sedov-Taylor blast wave.

For the supernova blastwave into the neutral ISM for the books, we set $j=3$ (spherical geometry), $\omega=0$ 
(uniform density), and $\gamma=5/3$ (atomic gas). 

In [ ]:
# Parameters for the calculation 

j = 3          # dimensionality index, j=1 planar, j=2 cylindrical, j=3 spherical
w = 0.         # initial density distribution power-law index, w=0 = uniform, w<j
gamma = 5./3.  # adiabatic exponent (ratio of specific heats) - mind the .'s!

# label for dimensionality aka geometry

geometry = ['planar','cylindrical','spherical']

# Are we in the singular case?

gSingular = (3.*j - 2. - w)/(j - 2.0 + w)  # Kamm Eqn 26

if gamma == gSingular:
    print(f'Warning: gamma={gamma:.2f} gives a singular solution (Kamm Eqn 26)')
    print(f'         Plot could be garbage or the computation blow up...')

# We will assume the standard solution, V2<V* prevails.  Beware if they give goofy input.

V0 = 2.0/((j+2.0-w)*gamma)  # Kamm Eqn 23

# Closed-form solution coefficients (Kamm Eqns 33-37)

a = ((j+2.-w)*(gamma+1.))/4.0
b = (gamma+1.)/(gamma-1.)
c = (gamma*(j+2.-w))/2.
d = ((j+2.-w)*(gamma+1.))/((j+2.-w)*(gamma+1.)-2.*(2.+j*(gamma-1.)))
e = (2+j*(gamma-1.))/2.0

# closed-form solution alpha parameters (Kamm Eqns 42-47).
# Note the order of evaluation matters since some alphas depend on
# others.

alpha0 = 2./(j+2.-w)
alpha2 = -(gamma-1.)/(2.*(gamma-1.)+j-(gamma*w))
alpha1 = (((j+2.-w)*gamma)/(2+j*(gamma-1.)))*((2*(j*(2.-gamma)-w)/(gamma*(j+2.-w)**2)) - alpha2)
alpha3 = (j-w)/(2*(gamma-1.)+j-(gamma*w))
alpha4 = (((j+2.-w)*(j-w))/(j*(2.-gamma)-w))*alpha1
alpha5 = (w*(1.+gamma)-2.*j)/(j*(2.-gamma)-w)

# Value of the similarity variable V at the origin (lam=0)

V0 = 2./((j+2.-w)*gamma)  # Eqn 23

# Value of V at the shock position:

V2 = 4.0/((j+2.-w)*(gamma+1.))    # Eqn 18

# Value of V at singular point

Vs = 2./((gamma-1.)*j + 2.)  # Eqn 19

# Post-shock density

# Compute the pre-shock conditions as a function of the similarity variable V

numSteps = 500

lS = []
fS = []
gS = []
hS = []

for V in np.linspace(V0,V2,numSteps):

    # Working variables x1..x4 for the standard and vacuum solutions (Kamm Eqn 29-32)

    x1 = a*V
    x2 = b*(c*V-1.)
    x3 = d*(1.-e*V)
    x4 = b*(1.-(c/gamma)*V)

    # dimensionless length lam = r/r2

    lam = (x1**(-alpha0))*(x2**(-alpha2))*(x3**(-alpha1))
    lS.append(lam)

    # v/v2 = f()

    f = x1*lam   # Sedov f()
    fS.append(f)

    # rho/rho2 = g()

    g = (x1**(w*alpha0))*(x2**(alpha3+w*alpha2))*(x3**(alpha4+w*alpha1))*(x4**(alpha5))  # Sedov g()
    gS.append(g)

    # p/p2 = h()

    h = (x1**(j*alpha0))*(x3**(alpha4+alpha1*(w-2)))*(x4**(1+alpha5))  # Sedov h()
    hS.append(h)

## Make the plot

Plot the pressure, density, and speed normalized relative to the immediate post-shock values:
 * pressure: $P/P_2$
 * density: $\rho/\rho_2$ and pre-shock $\rho_1/\rho_2$
 * speed: $u/u_2$ 

as a function of radius normalized by the location of the shock, $r/r_{sh}$.

In [ ]:
fig,ax = plt.subplots()

fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

# Limits

plt.xlim(0.0,1.5)
ax.xaxis.set_major_locator(MultipleLocator(0.2))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
plt.xlabel(r'$r/r_{\rm sh}$')

plt.ylim(0.0,1.1)
ax.yaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.2))
#ax.set_yticklabels([])

# Pressure plot 

plt.plot(lS,hS,color='black',ls='--',lw=2.0*lwidth)
plt.plot([1.0,1.0],[1.0,0.0],':',color='black',lw=2.0*lwidth)
plt.text(0.15,0.306,r'$P/P_2$',va="bottom",ha="center",color='black')
pp2str = f'{hS[0]:.3f}'
plt.text(-0.01,hS[0],pp2str,va="center",ha="right",color='black',fontsize=axisFontSize)

# velocity plot 

plt.plot(lS,fS,color='black',ls=':',lw=2.0*lwidth)
plt.plot([1.0,1.0],[1.0,0.0],ls=':',color='black',lw=2.0*lwidth)
plt.text(0.73,0.65,r'$u/u_2$',va="center",ha="right",color='black',fontsize=axisFontSize)

# density plot - black

rho12 = (gamma-1.0)/(gamma+1.0)
plt.plot(lS,gS,color='black',lw=2.0*lwidth)
plt.plot([1.0,1.0,1.3],[1.0,rho12,rho12],color='black',lw=2.0*lwidth)
plt.text(0.75,0.15,r'$\rho/\rho_2$',va="center",ha="right",color='black',fontsize=axisFontSize)
plt.text(1.32,rho12,r'$\rho_1/\rho_2$',va="center",ha="left",color='black',fontsize=axisFontSize)

# Label with gamma, should get fancier for omega and j...

if w == 0:
    density = rf'$\rho_1$=const'
else:
    density = rf'$\rho_1\propto r^{-{int(w)}}$'
    
if gamma==5./3.:
    plotLab = rf'{geometry[j-1]}, $\gamma=5/3$, {density}'
elif gamma==7./5.:
    plotLab = rf'{geometry[j-1]}, $\gamma=7/5$, {density}'
else:
    plotLab = rf'{geometry[j-1]}, $\gamma={gamma:.2f}$, {density}'
    
plt.text(0.1,0.9,plotLab,color='black',va='center',ha='left')

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')